<a href="https://colab.research.google.com/github/tonystz/cuda/blob/main/poc/gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Get code

In [ ]:
!git clone https://github.com/tonystz/cuda.git

In [2]:
!python cuda/poc/raw.py

[step0]preprocess raw log: total=1153965 skip=12507 2.091172933578491



###CPU

In [3]:
!python cuda/poc/cpu.py

[step1]reload data: 3.3212413787841797 (1141458,) |S44 1141458 [b'::1-200' b'::1-200' b'::1-404' ... b'180.163.28.55-200'
 b'180.163.28.55-200' b'180.163.28.55-200']
[step2]404 data filter: 0.6432945728302002 202448
[step3]check result: 0.17370939254760742 ['216.244.66.200', '217.174.192.132', '66.249.79.234', '66.249.79.238', '66.249.79.236', '23.96.16.211', '47.101.154.149', '47.101.149.21', '38.54.24.118']


###GPU initialize

In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

###GPU

In [ ]:
a=np.loadtxt('pre.log', delimiter=' ', dtype=np.string_,converters={0:lambda v:v+b'\0'})
print(a.shape,a.size,a)

In [21]:
print(a[0])

b'::1-200'


In [12]:
na=np.delete(a,0)
na.reshape((479,2383))

array([[b'::1-200', b'::1-404', b'::1-200', ..., b'195.242.110.253-200',
        b'110.249.202.11-200', b'110.249.201.69-200'],
       [b'110.249.201.206-200', b'207.46.13.23-200',
        b'110.249.201.14-200', ..., b'223.104.68.50-200',
        b'223.104.68.50-200', b'223.104.68.50-200'],
       [b'223.104.68.50-200', b'223.104.68.50-200', b'223.104.68.50-200',
        ..., b'125.84.82.12-200', b'125.84.82.12-200',
        b'125.84.82.12-200'],
       ...,
       [b'140.250.200.255-404', b'140.250.200.255-404',
        b'52.167.144.32-200', ..., b'180.164.68.86-200',
        b'180.164.68.86-200', b'180.164.68.86-200'],
       [b'180.164.68.86-200', b'180.164.68.86-200', b'180.164.68.86-200',
        ..., b'36.46.86.9-200', b'36.46.86.9-200', b'36.46.86.9-200'],
       [b'36.46.86.9-200', b'36.46.86.9-200', b'36.46.86.9-200', ...,
        b'180.163.28.55-200', b'180.163.28.55-200', b'180.163.28.55-200']],
      dtype='|S44')

In [15]:
s= np.array(['180.163.28.55-200\0','edfg\0','stz12345678b\0','b1\0','k2\0','hellworld3\0'], dtype=np.string_)

s=s.reshape(2,3)
print('shape:',s.shape,s.size)
print(s,s.dtype)

shape: (2, 3) 6
[[b'180.163.28.55-200' b'edfg' b'stz12345678b']
 [b'b1' b'k2' b'hellworld3']] |S18


In [93]:
%%writefile gpu.py
#!python 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np

mod = SourceModule("""
    #include <arpa/inet.h>
    const int LOG_LEN=44;
    __device__ void mystrcpy(char *in,char *out){
      while(*in != 0){
        *out++ = *in++;
      }
      *out = '\0';
    }
    __device__  int find_404_ipaddr(char*in_gpu,char outAddr[],int rowStartIndex ,int strStartIndex){
        int index=0;
        for(int i = 0; i<LOG_LEN; i++){
            index = strStartIndex+i+rowStartIndex;
            if (in_gpu[index]=='-'){
              if (in_gpu[index+1] == '4' && in_gpu[index+2] == '0' && in_gpu[index+3] == '4'){
                 outAddr[i]='\0';
                 return 1;
              }
            }
            outAddr[i]=in_gpu[index];
        }
        return 0;
    }

    __global__ void check_log(char*in_gpu,char* out_gpu)
    { 
      const int colLen=2383;
      const int rowCnt=479;

      int idx = threadIdx.x + blockIdx.x * blockDim.x;

      //printf("index: threadIdx.x=%d blockIdx.x=%d  blockDim.x=%d \\n",threadIdx.x,blockIdx.x,blockDim.x);
     // printf("thread id: [%d]  get str:%s  strlen=%d \\n",idx,in_gpu+LOG_LEN+1, LOG_LEN);

      // string len is 6
      char sub[LOG_LEN+1];
      int strStartIndex=0;//string start locaion
      int rowOffset=colLen*LOG_LEN;
      if(idx > rowCnt){
        printf("[ERROR]idx=%d, rowCnt=%d, we want one thread process all one rows'data .please tune");
      }
      
      //char *ipAddr=0;
      int rowStartIndex=rowOffset*idx;
      out_gpu[rowStartIndex]='\0';
      for(int c=0;c<colLen;c++){
        strStartIndex=c*LOG_LEN;
        if (find_404_ipaddr(in_gpu,sub,rowStartIndex,strStartIndex)){
           //printf("thread[%d] rowStartIndex=%d, substring:%s strStartIndex %d\\n",idx,rowStartIndex,sub,strStartIndex);

           //copy 404 to out_gpu
           mystrcpy(sub,out_gpu+rowStartIndex);
        }
       
      }
    }
    """)

check_log = mod.get_function("check_log")

a=np.loadtxt('pre.log', delimiter=' ', dtype=np.string_,converters={0:lambda v:v+b'\0'})
na=np.delete(a,0)
na.reshape((479,2383))
print('shape:',na.shape,na.size)
print(na,na.data)

NUM_ROW=479
NUM_COL=2383

na_gpu = gpuarray.to_gpu(na)
out_gpu = gpuarray.empty_like(na_gpu)
check_log(na_gpu,out_gpu,block=(NUM_ROW,1,1),grid=(1,1,1))
print('modify:',out_gpu.get())

Overwriting gpu.py


In [94]:
!python gpu.py

/content/gpu.py:8: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu:9:14: warning: null character(s) preserved in literal
    9 |       *out = ' ';
      |              ^
kernel.cu:17:29: warning: null character(s) preserved in literal
   17 |                  outAddr[i]=' ';
      |                             ^
kernel.cu:46:30: warning: null character(s) preserved in literal
   46 |       out_gpu[rowStartIndex]=' ';
      |                              ^
kernel.cu(9): warning #1193-D: null (zero) character in string or character constant

kernel.cu(17): warning #1193-D: null (zero) character in string or character constant

kernel.cu(41): warning #224-D: the format string requires additional arguments

kernel.cu(46): warning #1193-D: null (zero) character in string or character constant


  mod = SourceModule("""
shape: (1141457,) 1141457
[b'::1-200' b'::1-404' b'::1-200' ... b'180.163.28.55-200'
 b'180.163.28.55-200' b'180.163.28.55-200'] <memory at 0x7f12

In [95]:
!grep '110.249.202.4' pre.log

110.249.202.4-404
110.249.202.46-404
110.249.202.41-404
110.249.202.48-200
110.249.202.47-200
110.249.202.46-200
110.249.202.43-200
110.249.202.45-200
110.249.202.49-200
110.249.202.48-200
110.249.202.48-200
110.249.202.46-200
110.249.202.44-200
110.249.202.47-200
110.249.202.47-200
110.249.202.48-200
110.249.202.42-200
110.249.202.49-200
110.249.202.47-200
110.249.202.42-200
110.249.202.48-200
110.249.202.43-404
110.249.202.47-200
110.249.202.48-200
110.249.202.41-404
110.249.202.4-200
110.249.202.4-200
110.249.202.40-404
110.249.202.42-404
110.249.202.40-200
110.249.202.4-404
110.249.202.49-404
110.249.202.41-200
110.249.202.41-200
110.249.202.46-200
110.249.202.40-404
110.249.202.49-404
110.249.202.48-200
110.249.202.48-200
110.249.202.48-404
110.249.202.40-200
110.249.202.41-200
110.249.202.42-404
110.249.202.41-200
110.249.202.44-200
110.249.202.45-200
110.249.202.49-200
110.249.202.46-200
110.249.202.46-200
110.249.202.48-200
110.249.202.43-200
110.249.202.43-200
110.249.202.48-2